In [4]:
import folium
import json
import requests
from datetime import datetime
import os
import pandas as pd
from census import Census
from us import states
import geopandas as gpd
import geojson
import numpy as np

In [5]:
# Variables
key = "1e1f8670c9944ee8269a8d7065315bcd2f3eb0c8"
state_code = "06"
county_code = "073"

## Getting and Parsing Jsons from retrieved from TravelTime Api in separate files
I have to get it from a separate file so I can separate each requests

TODO: Make a function that lets you choose the stations you want to get the isochrones from instead of hard coded stations

In [6]:
# list all geojson files in blue_line folder that contain the word 'walking'
paths_walking = [os.path.join('blue_line', fn) for fn in next(os.walk('blue_line'))[2] if 'walking' in fn]


blue_stations_walking = []
# save JSON to variable
for i in range(len(paths_walking)):
    with open(paths_walking[i]) as f:
        data = json.load(f)
        blue_stations_walking.append(data)

# repeat for bike
paths_bike = [os.path.join('blue_line', fn) for fn in next(os.walk('blue_line'))[2] if 'cycling' in fn]

blue_stations_bike = []
for i in range(len(paths_bike)):
    with open(paths_bike[i]) as f:
        data = json.load(f)
        blue_stations_bike.append(data)

## Defining Function To Union GeoJsons as TravelTime only allows unions up to 10 at a time

In [7]:
import shapely.geometry as sg
import geojson

# list of the biking geojsons geometries
def union_stations(stations):
    poly = sg.Polygon()
    for station in stations:
        for i in range(len(station['features'])):
            if "last station: " in station['features'][i]['properties']['search_id']:
                poly = poly.union(sg.shape(station['features'][i]['geometry']))


    # Convert the unioned polygon into a GeoJSON Feature
    union_feature = geojson.Feature(geometry=poly, properties={})

    # Create a FeatureCollection with just the union feature
    union_feature_collection = geojson.FeatureCollection([union_feature])

    # Convert the FeatureCollection to a GeoJSON string
    union_geojson_str = geojson.dumps(union_feature_collection)

    return union_geojson_str

## Getting US Census API data for population/inhabitants in each isochrone

In [8]:
def get_census_data(state_code, county_code, API_KEY):
    """
    Pull the census data for a given state and county and return a dataframe
    """
    url = f'https://api.census.gov/data/2020/dec/pl?get=NAME,H1_001N&for=block:*&in=state:{state_code}&in=county:{county_code}&key={API_KEY}'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

def get_density_by_bg(df, gdf, transport_type_stations, county):
    # Split the "Block Group" from the NAME column into its own column
    df['Block Group'] = df['NAME'].str.split(',', expand=True)[1]

    # Convert the population column to an integer
    df['H1_001N'] = df['H1_001N'].astype(int)

    # groupby tract then block group and sum the population
    df = df.groupby(['state', 'county', 'tract', 'Block Group'])['H1_001N'].sum().reset_index()

    # remove the last row
    df = df[:-1]
    tracts = df['tract'].unique()

    df['Block Group'] = df['Block Group'].str.replace('Block Group ', '')
    df['H1_001N'] = df['H1_001N'].astype(str)

    # ----------------------------
    path = f'tracts_geo_json/bg/bg_{county}.geojson'

    # get all tracts in df that are in tracts
    temp = gdf[(gdf['TRACTCE'].isin(tracts)) & (gdf['COUNTYFP'] == '073')].sort_values(by='TRACTCE')

    # add population to geojson
    temp['H1_001N'] = df['H1_001N'].values

    # join Tract and Block Group to create a unique identifier
    temp['tract'] = temp['TRACTCE'] + temp['BLKGRPCE']

    # get all polygons that intersect with the union of the biking geojsons
    temp = temp[temp.intersects(sg.shape(geojson.loads(union_stations(transport_type_stations))[0]['geometry']))]

    # Save temp as a geojson to a variable
    geojson_str = temp.to_json()

    # ----------------------------

    data_df = pd.DataFrame()
    data_df['GEOID'] = temp['GEOID'].values
    data_df['H1_001N'] = temp['H1_001N'].values

    # convert to int
    data_df['H1_001N'] = data_df['H1_001N'].astype(int)

    return data_df, geojson_str

In [9]:
# Read in shapefile
shp_file = r'CA_shape_file\bg\cb_2021_06_bg_500k.shp'
gdf = gpd.read_file(shp_file)

In [10]:
def update_shapes(gs, transport_poly, df):
    percents = []
    df = df.copy()
    # Get the Polygon of the area covered by transport
    intersection = sg.shape(geojson.loads(transport_poly)[0]['geometry'])
    for feature in gs['features']:
        # Get the intersection of the BG and the transport area
        geometry = sg.shape(feature['geometry']).intersection(intersection)

        # Calculate the percentage of the BG that is covered by the transport area
        inter_percent = geometry.area/sg.shape(feature['geometry']).area
        percents.append(inter_percent)
        # Merge the area if the polygon is a multipolygon
        if isinstance(geometry, sg.MultiPolygon):
            geometry = geometry.buffer(0)  # merge MultiPolygons into a single Polygon
        feature['geometry'] = sg.mapping(geometry)
    
    # Update values on df
    df['H1_001N'] = df['H1_001N'] * percents
    # Update values in gs
    for i in range(len(gs['features'])):
        gs['features'][i]['properties']['H1_001N'] = df['H1_001N'].values[i] * percents[i]

    return gs, df


In [11]:
blue_walk_poly = union_stations(blue_stations_walking)
blue_census_data = get_census_data(state_code, county_code, key)
blue_data_df_walk, blue_gs_walk_str = get_density_by_bg(blue_census_data, gdf, blue_stations_walking, 'SD')

blue_gs_walk = geojson.loads(blue_gs_walk_str)
blue_gs_walk, blue_data_df_walk  = update_shapes(blue_gs_walk, blue_walk_poly, blue_data_df_walk)

In [12]:
blue_bike_poly = union_stations(blue_stations_bike)
blue_census_data = get_census_data(state_code, county_code, key)
blue_data_df_bike, blue_gs_bike_str = get_density_by_bg(blue_census_data, gdf, blue_stations_bike, 'SD')

blue_gs_bike = geojson.loads(blue_gs_bike_str)
blue_gs_bike, blue_data_df_bike  = update_shapes(blue_gs_bike, blue_bike_poly, blue_data_df_bike)

In [13]:
lat = 32.86926550131975
lng = -117.21402645890504
thresholds = [0, 250, 500, 750, 1000, 1250, 2000]
tooltip = folium.GeoJsonTooltip(fields=['GEOID', 'H1_001N'])
map_obj = folium.Map(location=[lat, lng], zoom_start=13)
# using gs as the geojson create a choropleth map with the population data for bike
blue_blue_cp = folium.Choropleth(
    geo_data=blue_gs_bike,
    name='Blue Line Bike Population',
    data=blue_data_df_bike,
    columns=['GEOID', 'H1_001N'],
    key_on='feature.properties.GEOID',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population',
    threshold_scale = thresholds,
    tooltip=tooltip,
    highlight=True
).add_to(map_obj)

folium.GeoJsonTooltip(['GEOID', 'H1_001N']).add_to(blue_blue_cp.geojson)


# add bike in red
blue_fg_bike = folium.FeatureGroup(name='Blue Line Bike')
folium.GeoJson(blue_bike_poly, style_function=lambda x: {'color':'red'}).add_to(blue_fg_bike)

# add feature groups to map
map_obj.add_child(blue_fg_bike)


In [14]:
# using gs as the geojson create a choropleth map with the population data for walking

blue_walk_cp = folium.Choropleth(
    geo_data=blue_gs_walk,
    name='Blue Line Walk Population',
    data=blue_data_df_walk,
    columns=['GEOID', 'H1_001N'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population',
    threshold_scale = thresholds,
    tooltip=tooltip,
    highlight=True,
).add_to(map_obj)

folium.GeoJsonTooltip(['GEOID', 'H1_001N']).add_to(blue_walk_cp.geojson)

# add walking in blue
blue_fg_walking = folium.FeatureGroup(name='Blue Line Walking')
folium.GeoJson(blue_walk_poly, style_function=lambda x: {'color':'blue'}).add_to(blue_fg_walking)

# add feature groups to map
map_obj.add_child(blue_fg_walking)

In [15]:
map_obj

In [16]:
print("The total number of people that can be reached by bike is: ", round(blue_data_df_bike['H1_001N'].sum()))
print("The total number of people that can be reached by walking is: ", round(blue_data_df_walk['H1_001N'].sum()))

The total number of people that can be reached by bike is:  127289
The total number of people that can be reached by walking is:  36632


# Green Line

In [17]:
# list all geojson files in blue_line folder that contain the word 'walking'
paths_walking = [os.path.join('green_line', fn) for fn in next(os.walk('green_line'))[2] if 'walking' in fn]


green_stations_walking = []
# save JSON to variable
for i in range(len(paths_walking)):
    with open(paths_walking[i]) as f:
        data = json.load(f)
        green_stations_walking.append(data)

# repeat for bike
paths_bike = [os.path.join('green_line', fn) for fn in next(os.walk('green_line'))[2] if 'cycling' in fn]

green_stations_bike = []
for i in range(len(paths_bike)):
    with open(paths_bike[i]) as f:
        data = json.load(f)
        green_stations_bike.append(data)

In [18]:
green_walk_poly = union_stations(green_stations_walking)
green_census_data = get_census_data(state_code, county_code, key)
green_data_df_walk, green_gs_walk_str = get_density_by_bg(green_census_data, gdf, green_stations_walking, 'SD')

green_gs_walk = geojson.loads(green_gs_walk_str)
green_gs_walk, green_data_df_walk  = update_shapes(green_gs_walk, green_walk_poly, green_data_df_walk)

green_bike_poly = union_stations(green_stations_bike)
green_census_data = get_census_data(state_code, county_code, key)
green_data_df_bike, green_gs_bike_str = get_density_by_bg(green_census_data, gdf, green_stations_bike, 'SD')

green_gs_bike = geojson.loads(green_gs_bike_str)
green_gs_bike, green_data_df_bike  = update_shapes(green_gs_bike, green_bike_poly, green_data_df_bike)

In [19]:
# using gs as the geojson create a choropleth map with the population data for bike
green_bike_cp = folium.Choropleth(
    geo_data=green_gs_bike,
    name='Green Line Bike Population',
    data=green_data_df_bike,
    columns=['GEOID', 'H1_001N'],
    key_on='feature.properties.GEOID',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population',
    threshold_scale = thresholds,
    tooltip=tooltip,
    highlight=True
).add_to(map_obj)

folium.GeoJsonTooltip(['GEOID', 'H1_001N']).add_to(green_bike_cp.geojson)


# add bike in red
green_fg_bike = folium.FeatureGroup(name='Green Line Bike')
folium.GeoJson(green_bike_poly, style_function=lambda x: {'color':'red'}).add_to(green_fg_bike)

# add feature groups to map
map_obj.add_child(green_fg_bike)

In [20]:
green_walk_cp = folium.Choropleth(
    geo_data=green_gs_walk,
    name='Green Line Walk Population',
    data=green_data_df_walk,
    columns=['GEOID', 'H1_001N'],
    key_on='feature.properties.GEOID',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population',
    threshold_scale = thresholds,
    tooltip=tooltip,
    highlight=True,
).add_to(map_obj)

folium.GeoJsonTooltip(['GEOID', 'H1_001N']).add_to(green_walk_cp.geojson)

# add walking in blue
green_fg_walking = folium.FeatureGroup(name='Green Line Walking')
folium.GeoJson(green_walk_poly, style_function=lambda x: {'color':'blue'}).add_to(green_fg_walking)

# add feature groups to map
map_obj.add_child(green_fg_walking)

# add layer control
folium.LayerControl().add_to(map_obj)

In [21]:
map_obj.save('map.html')

In [22]:
print("The total number of people that can be reached by bike is: ", round(green_data_df_bike['H1_001N'].sum()))
print("The total number of people that can be reached by walking is: ", round(green_data_df_walk['H1_001N'].sum()))

The total number of people that can be reached by bike is:  100231
The total number of people that can be reached by walking is:  24328
